In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import neuralnetwork as nn
from tfomics import neuralbuild as nb
from tfomics import utils, learn, init
from tfomics import visualize
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def load_data(data_path):
    dataset = h5py.File(data_path,'r')

    print("loading training data")
    X_train = np.expand_dims(dataset['X_train'], axis=3).transpose([0,2,3,1])
    y_train = np.array(dataset['Y_train'])
    index = np.where(y_train[:,0]==0)[0]
    y = np.zeros((y_train.shape[0],1))
    y[index] = 1
    y_train = np.hstack([y_train, y])
    
    print("loading validation data")  
    X_valid = np.expand_dims(dataset['X_valid'], axis=3).transpose([0,2,3,1])
    y_valid = np.array(dataset['Y_valid'])
    index = np.where(y_valid[:,0]==0)[0]
    y = np.zeros((y_valid.shape[0],1))
    y[index] = 1
    y_valid = np.hstack([y_valid, y])

    print("loading test data")
    X_test = np.expand_dims(dataset['X_test'], axis=3).transpose([0,2,3,1])
    y_test = np.array(dataset['Y_test'])
    index = np.where(y_test[:,0]==0)[0]
    y = np.zeros((y_test.shape[0],1))
    y[index] = 1
    y_test = np.hstack([y_test, y])

    return X_train, y_train, X_valid, y_valid, X_test, y_test 


data_path = '/Users/juliankimura/Desktop/genome'
file_name = 'K562_CEBPB_200_genome.h5'
X_train, y_train, X_valid, y_valid, X_test, y_test = load_data(os.path.join(data_path, file_name))

loading training data
loading validation data
loading test data


In [3]:

def model(input_shape, num_labels=None):

    # create model
    layer1 = {'layer': 'input',
            'input_shape': input_shape
            }
    layer2 = {'layer': 'conv1d', 
            'num_filters': 32,
            'filter_size': 19,
            'norm': 'batch',
            'activation': 'relu',
            'pool_size': 25,
            'padding': 'SAME',
            }
    layer3 = {'layer': 'conv1d_residual',
            'filter_size': 5,
            'stochastic': 1.0,
           }    
    layer4 = {'layer': 'dense', 
            'num_units': 48,
            #'norm': 'batch',
            'activation': 'relu',
            'dropout': 0.2
            }  
    layer5 = {'layer': 'dense', 
            'num_units': num_labels,
            'activation': 'softmax'
            }

    #from tfomics import build_network
    model_layers = [layer1, layer2, layer3, layer4, layer5]
    
    # optimization parameters
    optimization = {"objective": "categorical",
                  "optimizer": "adam",
                  "learning_rate": 0.001,      
                  "l2": 1e-6,
                  # "l1": 0, 
                  }
    return model_layers, optimization
# get shapes
num_data, height, width, dim = X_train.shape
input_shape=[None, height, width, dim]
num_labels = y_train.shape[1]  
model_layers, optimization = model(input_shape, num_labels)

model_layers, optimization = model(input_shape, num_labels)
nnbuild = nb.NeuralBuild(model_layers)
network, placeholders, hidden_feed_dict = nnbuild.get_network_build()

In [4]:

# build neural network class
nnmodel = nn.NeuralNet(network, placeholders, hidden_feed_dict)
nnmodel.inspect_layers()

# set output file paths

results_path = utils.make_directory(data_path, 'results')
results_path = utils.make_directory(results_path, 'tfomics')
output_name = 'test'
filepath = os.path.join(results_path, output_name)

# compile neural trainer
nntrainer = nn.NeuralTrainer(nnmodel, optimization, save='best', filepath=filepath)

----------------------------------------------------------------------------
Network architecture:
----------------------------------------------------------------------------
layer1: inputs
(?, 200, 1, 4)
layer2: conv1d_0
(?, 200, 1, 32)
layer3: conv1d_0_batch
(?, 200, 1, 32)
layer4: conv1d_0_active
(?, 200, 1, 32)
layer5: conv1d_0_pool
(?, 8, 1, 32)
layer6: conv1d_residual_0_conv1d_resid_stochastic
(?, 8, 1, 32)
layer7: dense_0
(?, 48)
layer8: dense_0_bias
(?, 48)
layer9: dense_0_active
(?, 48)
layer10: dense_0_dropout
(?, 48)
layer11: dense_1
(?, 2)
layer12: dense_1_bias
(?, 2)
layer13: output
(?, 2)
----------------------------------------------------------------------------


In [5]:
train = {'inputs': [X_train], 'targets': y_train}
valid = {'inputs': [X_valid], 'targets': y_valid}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=200, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

Epoch 1 out of 50 


FailedPreconditionError: Attempting to use uninitialized value cond_2/cond/Variable
	 [[Node: cond_2/cond/Variable/read = Identity[T=DT_FLOAT, _class=["loc:@cond_2/cond/Variable"], _device="/job:localhost/replica:0/task:0/cpu:0"](cond_2/cond/Variable)]]

Caused by op 'cond_2/cond/Variable/read', defined at:
  File "/Users/juliankimura/anaconda/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/juliankimura/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/juliankimura/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/juliankimura/anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/juliankimura/anaconda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/juliankimura/a

In [10]:
train = {'inputs': [X_train], 'targets': y_train}
valid = {'inputs': [X_valid], 'targets': y_valid}
data = {'train': train, 'valid': valid}
learn.train_minibatch(nntrainer, data, batch_size=200, num_epochs=50, 
                    patience=10, verbose=2, shuffle=True)

Epoch 1 out of 50 


FailedPreconditionError: Attempting to use uninitialized value cond_2/cond/Variable
	 [[Node: cond_2/cond/Variable/read = Identity[T=DT_FLOAT, _class=["loc:@cond_2/cond/Variable"], _device="/job:localhost/replica:0/task:0/cpu:0"](cond_2/cond/Variable)]]

Caused by op 'cond_2/cond/Variable/read', defined at:
  File "/Users/juliankimura/anaconda/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/juliankimura/anaconda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/juliankimura/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/juliankimura/anaconda/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/juliankimura/anaconda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/juliankimura/a